# Using pre toknized binidx/numpy datasets with infctx trainer

**THIS IS AN INCOMPLETE NOTEBOOK TUTORIAL**

The following is the backward compatibility support for various binary dataset formats

## Intial setup

Before we go into the dataset setup, lets perform an initial setup for all the folders we need, and a small toy model which we would use throughout the various examples within this notebook.

In [7]:
# Setup the folders we will need
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/

# Initialized a simple L6-D512 model, for both the v4 neox (50277) tokenizer
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size neox ../model/L6-D512-neox-init.pth

# and rwkv world (65529) tokenizers
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size world ../model/L6-D512-world-init.pth

# If you have a custom vocab size, you can indicate accordingly as well with an int
!cd ../../RWKV-v4neo/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size 20259 ../model/L6-D512-V20259-init.pth

[2023-07-29 07:11:05,106] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230706'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-neox-init.pth
Vocab size: 50277
---- ----- ----
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_2048_bf16/build.ninja...
Building extension module wkv_2048_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_2048_bf16...
50277 512   -0.1 emb.weight
512   512   0    blocks.0.att.key.weight
512   512   1.0  blocks.0.att.value.weight
512   512   0    blocks.0.att.receptance.weight
512   512   0    blocks.0.att.output.weigh

## Training using a binidx dataset

The following is the `binidx-enwiki.yaml` settings, for using a textual dataset via huggingface, with all the comments removed

```.yaml

```

In [27]:
# Setup the dataset dir
!mkdir -p ../../dataset/dataset-config/

# Download the binidx file
!cd ../../dataset/dataset-config/ && wget -nc https://huggingface.co/datasets/picocreator/RWKV-notebook-assets/resolve/main/sample_data_text_document.bin
!cd ../../dataset/dataset-config/ && wget -nc https://huggingface.co/datasets/picocreator/RWKV-notebook-assets/resolve/main/sample_data_text_document.idx



File ‘sample_data_text_document.bin’ already there; not retrieving.



File ‘sample_data_text_document.idx’ already there; not retrieving.



In [1]:
# Lets preload the requried dataset
!cd ../../RWKV-v4neo && python3 preload_dataset.py ../notebook/dataset-config/example-binidx.yaml

Dataset generator downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/generator/default-a5618f1469c092fc/0.0.0. Subsequent calls will reuse this data.


In [3]:
# Train using the converted binidx format
!cd ../../RWKV-v4neo && python3 new_train.py fit -c ../notebook/dataset-config/example-binidx.yaml

[2023-07-30 04:55:19,943] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230706'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1003332801
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1003332801
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/connector.py:555: Use